# BP-SOM for BERT Fine-tuning on SST-2

This notebook implements BP-SOM (Back-Propagation with Self-Organizing Maps) for BERT fine-tuning on the SST-2 sentiment analysis task.

**BP-SOM** combines supervised backpropagation with unsupervised SOM clustering:
- Standard BP error from task loss
- SOM error from class-specific clustering
- Combined learning: `error = (1-α) * bp_error + α * som_error`

## Setup

Run this notebook on a GPU-enabled environment (Colab, Kaggle, etc.) for faster training.

In [ ]:
# Install dependencies
!pip install -q torch transformers datasets numpy scipy scikit-learn matplotlib seaborn tqdm

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import (
    BertTokenizer,
    BertModel,
    BertConfig,
    BertPreTrainedModel,
    BertForSequenceClassification,
    get_linear_schedule_with_warmup
)
from transformers.modeling_outputs import SequenceClassifierOutput
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from typing import Optional, Tuple, Dict, List

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Self-Organizing Map (SOM) Layer

Implements the core SOM algorithm for BP-SOM.

In [ ]:
class SelfOrganizingMap(nn.Module):
    """
    Self-Organizing Map for BP-SOM architecture.
    Optimized with vectorized operations for GPU acceleration.
    """

    def __init__(
        self,
        input_dim: int,
        grid_size: int,
        num_classes: int,
        som_lr_max: float = 0.20,
        som_lr_min: float = 0.05,
        som_context_max: int = 2,
        som_context_min: int = 0,
        reliability_threshold: float = 0.95,
        som_error_weight: float = 0.25,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.grid_size = grid_size
        self.num_classes = num_classes
        self.som_lr_max = som_lr_max
        self.som_lr_min = som_lr_min
        self.som_context_max = som_context_max
        self.som_context_min = som_context_min
        self.reliability_threshold = reliability_threshold
        self.som_error_weight = som_error_weight

        # SOM grid storage
        self.register_buffer(
            'som_vectors',
            torch.rand(grid_size, grid_size, input_dim) * 0.5 + 0.5
        )
        self.register_buffer(
            'cell_class_counts',
            torch.zeros(grid_size, grid_size, num_classes + 1, dtype=torch.long)
        )
        self.register_buffer(
            'cell_labels',
            torch.zeros(grid_size, grid_size, dtype=torch.long)
        )
        self.register_buffer(
            'cell_reliability',
            torch.zeros(grid_size, grid_size, dtype=torch.float)
        )

        self.total_distance = 0.0
        self.som_usage_count = 0
        self.total_examples = 0

    def get_distance_matrix(self, x: torch.Tensor, w: torch.Tensor) -> torch.Tensor:
        """
        Compute squared Euclidean distance matrix between batch x and vectors w.
        x: (B, D), w: (N, D) -> returns (B, N)
        """
        x_norm = (x**2).sum(1).view(-1, 1)
        w_norm = (w**2).sum(1).view(1, -1)
        dist = x_norm + w_norm - 2.0 * torch.mm(x, w.t())
        return torch.clamp(dist, 0.0, None)

    def update_som_network_batch(
        self,
        activations: torch.Tensor,
        bmu_indices: torch.Tensor,
        som_lr: float,
        som_context: int
    ):
        """Update SOM prototypes using batch neighborhood learning."""
        B = activations.size(0)
        G = self.grid_size

        # Convert indices to coordinates
        bmu_x = bmu_indices // G
        bmu_y = bmu_indices % G

        # Create coordinate grid (G, G)
        y_grid, x_grid = torch.meshgrid(
            torch.arange(G, device=activations.device),
            torch.arange(G, device=activations.device),
            indexing='ij'
        )

        # Compute Chebyshev distances for all grid cells to all BMUs
        # Expand to (B, G, G)
        dist_x = torch.abs(x_grid.unsqueeze(0) - bmu_x.view(B, 1, 1))
        dist_y = torch.abs(y_grid.unsqueeze(0) - bmu_y.view(B, 1, 1))
        dist_chebyshev = torch.max(dist_x, dist_y)

        # Compute update powers
        mask = (dist_chebyshev <= som_context).float()
        powers = som_lr / (2.0 ** dist_chebyshev)
        powers = powers * mask # (B, G, G)

        # Accumulate updates: sum_k (p_k * (x_k - w))
        # W_new = W_old + sum_k(p_k * x_k) - W_old * sum_k(p_k)

        denom = powers.sum(dim=0).unsqueeze(-1) # (G, G, 1)

        powers_flat = powers.view(B, -1) # (B, G*G)
        num_flat = torch.matmul(powers_flat.t(), activations) # (G*G, B) x (B, D) -> (G*G, D)
        numerator = num_flat.view(G, G, -1)

        self.som_vectors += (numerator - self.som_vectors * denom)

    def forward(
        self,
        activations: torch.Tensor,
        labels: Optional[torch.Tensor] = None,
        epoch: int = 0,
        max_epochs: int = 100,
        training: bool = True
    ) -> Tuple[Optional[torch.Tensor], dict]:
        """Process activations through SOM using vectorized operations."""
        batch_size = activations.size(0)

        # Compute current SOM parameters
        progress = (max_epochs - epoch) / max_epochs if max_epochs > 0 else 0
        progress = max(0, min(1, progress))

        som_context = self.som_context_min + int(
            (progress ** 4) * (self.som_context_max - self.som_context_min)
        )
        som_lr = self.som_lr_min + (progress ** 1) * (self.som_lr_max - self.som_lr_min)

        # Flatten SOM vectors for distance computation: (G*G, D)
        flat_som = self.som_vectors.view(-1, self.input_dim)

        # 1. Compute Distances and Global BMUs
        dists = self.get_distance_matrix(activations, flat_som) # (B, G*G)
        min_dists, bmu_indices = torch.min(dists, dim=1) # (B,)

        # Statistics
        total_dist = torch.sqrt(min_dists).sum().item()
        som_used_count = 0
        som_errors = None

        if training and labels is not None:
            # 2. Compute Class-Specific BMUs and Errors
            # Mask distances where cell_label != label
            flat_labels = self.cell_labels.view(-1)
            # (B, G*G) mask
            label_mask = (flat_labels.unsqueeze(0) != labels.unsqueeze(1))

            class_dists = dists.clone()
            class_dists[label_mask] = float('inf')

            _, class_bmu_indices = torch.min(class_dists, dim=1)

            # Get prototypes and reliability for class BMUs
            class_bmu_vectors = flat_som[class_bmu_indices] # (B, D)
            flat_reliability = self.cell_reliability.view(-1)
            reliabilities = flat_reliability[class_bmu_indices] / 100.0 # (B,)

            # Compute SOM errors
            # valid if reliability >= threshold
            valid_mask = (reliabilities >= self.reliability_threshold).float().unsqueeze(1)
            som_errors = 0.01 * reliabilities.unsqueeze(1) * (class_bmu_vectors - activations)
            som_errors = som_errors * valid_mask

            som_used_count = (reliabilities >= self.reliability_threshold).sum().item()

            # 3. Update SOM Network
            if min_dists.mean() > 1e-4:
                self.update_som_network_batch(activations, bmu_indices, som_lr, som_context)

        self.total_distance += total_dist
        self.som_usage_count += som_used_count
        self.total_examples += batch_size

        stats = {
            'som_lr': som_lr,
            'som_context': som_context,
            'avg_distance': total_dist / batch_size if batch_size > 0 else 0,
            'som_usage_pct': (som_used_count / batch_size * 100) if batch_size > 0 else 0,
        }

        return som_errors, stats

    def update_cell_labels(self, activations: torch.Tensor, labels: torch.Tensor):
        """Update SOM cell class labels using batched processing."""
        self.cell_class_counts.zero_()

        flat_som = self.som_vectors.view(-1, self.input_dim)
        batch_size = 1024 # Process in chunks to save memory

        with torch.no_grad():
            for i in range(0, activations.size(0), batch_size):
                batch_act = activations[i:i+batch_size]
                batch_lbl = labels[i:i+batch_size]

                dists = self.get_distance_matrix(batch_act, flat_som)
                _, bmu_indices = torch.min(dists, dim=1)

                bmu_x = bmu_indices // self.grid_size
                bmu_y = bmu_indices % self.grid_size

                # Accumulate counts
                for j, label in enumerate(batch_lbl):
                    self.cell_class_counts[bmu_x[j], bmu_y[j], label] += 1

        # Update labels and reliability
        for x in range(self.grid_size):
            for y in range(self.grid_size):
                counts = self.cell_class_counts[x, y]
                total = counts.sum().item()

                if total > 0:
                    max_count, max_class = counts.max(0)
                    self.cell_labels[x, y] = max_class
                    self.cell_reliability[x, y] = (max_count.float() / total) * 100
                else:
                    self.cell_labels[x, y] = 0
                    self.cell_reliability[x, y] = 0

    def reset_statistics(self):
        """Reset epoch statistics."""
        self.total_distance = 0.0
        self.som_usage_count = 0
        self.total_examples = 0

    def get_statistics(self) -> dict:
        """Get accumulated statistics."""
        return {
            'avg_distance': self.total_distance / max(1, self.total_examples),
            'som_usage_pct': (self.som_usage_count / max(1, self.total_examples)) * 100,
            'total_examples': self.total_examples,
        }

print("✓ Vectorized SOM Layer defined")

## 2. BP-SOM BERT Model

Integrates BERT with BP-SOM hidden layer.

In [ ]:
class BPSOMHiddenLayer(nn.Module):
    """Hidden layer with attached SOM for BP-SOM learning."""

    def __init__(
        self,
        input_dim: int,
        hidden_dim: int,
        num_classes: int,
        dropout: float = 0.1,
        som_grid_size: int = 20,
        som_lr_max: float = 0.20,
        som_lr_min: float = 0.05,
        som_context_max: int = 2,
        som_context_min: int = 0,
        reliability_threshold: float = 0.95,
        som_error_weight: float = 0.25,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.weight = nn.Parameter(torch.randn(hidden_dim, input_dim) * 0.1)
        self.bias = nn.Parameter(torch.zeros(hidden_dim))
        self.activation = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout)

        self.som = SelfOrganizingMap(
            input_dim=hidden_dim,
            grid_size=som_grid_size,
            num_classes=num_classes,
            som_lr_max=som_lr_max,
            som_lr_min=som_lr_min,
            som_context_max=som_context_max,
            som_context_min=som_context_min,
            reliability_threshold=reliability_threshold,
            som_error_weight=som_error_weight,
        )

        self.som_error_weight = som_error_weight

        self.register_buffer('activation_sum', torch.zeros(hidden_dim))
        self.register_buffer('activation_sum_sq', torch.zeros(hidden_dim))
        self.register_buffer('activation_count', torch.zeros(1))

    def forward(
        self,
        x: torch.Tensor,
        labels: Optional[torch.Tensor] = None,
        epoch: int = 0,
        max_epochs: int = 100,
        training: bool = True
    ) -> Tuple[torch.Tensor, dict]:
        """Forward pass with SOM processing."""
        z = torch.nn.functional.linear(x, self.weight, self.bias)
        activations = self.activation(z)

        if training:
            with torch.no_grad():
                self.activation_sum += activations.sum(dim=0)
                self.activation_sum_sq += (activations ** 2).sum(dim=0)
                self.activation_count += activations.size(0)

        som_errors, som_stats = self.som.forward(
            activations,
            labels=labels,
            epoch=epoch,
            max_epochs=max_epochs,
            training=training
        )

        if training and labels is not None and som_errors is not None:
            def som_gradient_hook(grad):
                bp_weight = 1.0 - self.som_error_weight
                combined_grad = bp_weight * grad + self.som_error_weight * som_errors
                return combined_grad

            activations.register_hook(som_gradient_hook)

        activations = self.dropout(activations)
        return activations, som_stats

    def get_activation_statistics(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get mean and std of activations."""
        if self.activation_count > 0:
            mean = self.activation_sum / self.activation_count
            variance = (self.activation_sum_sq / self.activation_count) - (mean ** 2)
            std = torch.sqrt(torch.clamp(variance, min=0))
        else:
            mean = torch.zeros_like(self.activation_sum)
            std = torch.zeros_like(self.activation_sum)
        return mean, std

    def reset_activation_statistics(self):
        """Reset activation tracking."""
        self.activation_sum.zero_()
        self.activation_sum_sq.zero_()
        self.activation_count.zero_()


class BPSOMBertForSequenceClassification(BertPreTrainedModel):
    """BERT model with BP-SOM hidden layer for sequence classification."""

    def __init__(self, config, bpsom_config=None):
        super().__init__(config)

        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config)

        if bpsom_config is None:
            bpsom_config = {
                'hidden_size': 128,
                'dropout': 0.1,
                'som_grid_size': 20,
                'som_lr_max': 0.20,
                'som_lr_min': 0.05,
                'som_context_max': 2,
                'som_context_min': 0,
                'reliability_threshold': 0.95,
                'som_error_weight': 0.25,
            }

        self.bpsom_hidden = BPSOMHiddenLayer(
            input_dim=config.hidden_size,
            hidden_dim=bpsom_config.get('hidden_size', 128),
            num_classes=config.num_labels,
            dropout=bpsom_config.get('dropout', 0.1),
            som_grid_size=bpsom_config.get('som_grid_size', 20),
            som_lr_max=bpsom_config.get('som_lr_max', 0.20),
            som_lr_min=bpsom_config.get('som_lr_min', 0.05),
            som_context_max=bpsom_config.get('som_context_max', 2),
            som_context_min=bpsom_config.get('som_context_min', 0),
            reliability_threshold=bpsom_config.get('reliability_threshold', 0.95),
            som_error_weight=bpsom_config.get('som_error_weight', 0.25),
        )

        self.classifier = nn.Linear(bpsom_config.get('hidden_size', 128), config.num_labels)

        self.post_init()

        self.current_epoch = 0
        self.max_epochs = 100

    def set_epoch(self, epoch: int, max_epochs: int):
        """Set current epoch for SOM parameter scheduling."""
        self.current_epoch = epoch
        self.max_epochs = max_epochs

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        return_dict: Optional[bool] = None,
    ) -> SequenceClassifierOutput:
        """Forward pass through BERT + BP-SOM + Classifier."""
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
        )

        cls_output = outputs.last_hidden_state[:, 0, :]

        hidden_output, som_stats = self.bpsom_hidden(
            cls_output,
            labels=labels,
            epoch=self.current_epoch,
            max_epochs=self.max_epochs,
            training=self.training
        )

        logits = self.classifier(hidden_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def update_som_labels(self, dataloader, device):
        """Update SOM cell labels based on training data."""
        all_activations = []
        all_labels = []

        self.eval()
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Updating SOM labels"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = self.bert(input_ids, attention_mask=attention_mask, return_dict=True)
                cls_output = outputs.last_hidden_state[:, 0, :]

                z = torch.nn.functional.linear(cls_output, self.bpsom_hidden.weight, self.bpsom_hidden.bias)
                activations = self.bpsom_hidden.activation(z)

                # Keep on device for GPU-accelerated SOM operations
                all_activations.append(activations)
                all_labels.append(labels)

        all_activations = torch.cat(all_activations, dim=0)
        all_labels = torch.cat(all_labels, dim=0)

        self.bpsom_hidden.som.update_cell_labels(all_activations, all_labels)
        self.train()

print("✓ BP-SOM BERT Model defined")

## 3. Data Loading

Load SST-2 dataset from HuggingFace.

In [ ]:
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load SST-2 dataset
print("Loading SST-2 dataset...")
dataset = load_dataset('glue', 'sst2')

print(f"Train examples: {len(dataset['train'])}")
print(f"Validation examples: {len(dataset['validation'])}")
print(f"Test examples: {len(dataset['test'])}")

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)

print("\nTokenizing...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare dataloaders
def collate_fn(batch):
    return {
        'input_ids': torch.tensor([item['input_ids'] for item in batch]),
        'attention_mask': torch.tensor([item['attention_mask'] for item in batch]),
        'labels': torch.tensor([item['label'] for item in batch])
    }

batch_size = 32

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

print("\n✓ Data loaded and prepared")

## 4. Training Functions

In [ ]:
def train_epoch(model, dataloader, optimizer, scheduler, device, epoch, max_epochs):
    """Train for one epoch."""
    model.train()
    model.set_epoch(epoch, max_epochs)

    total_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{max_epochs} [Train]")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

        progress_bar.set_postfix({
            'loss': f"{loss.item():.4f}",
            'acc': f"{correct/total*100:.2f}%"
        })

    return {
        'loss': total_loss / len(dataloader),
        'accuracy': correct / total * 100
    }


def eval_epoch(model, dataloader, device, desc="Eval"):
    """Evaluate model."""
    model.eval()

    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=desc):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            total_loss += outputs.loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return {
        'loss': total_loss / len(dataloader),
        'accuracy': correct / total * 100
    }

print("✓ Training functions defined")

## 5. Visualization Functions

In [ ]:
def plot_som_visualization(model, class_names=['negative', 'positive']):
    """Plot SOM cell labels and reliability."""
    som = model.bpsom_hidden.som
    labels = som.cell_labels.cpu().numpy()
    reliability = som.cell_reliability.cpu().numpy()
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Labels
    sns.heatmap(
        labels,
        annot=True,
        fmt='d',
        cmap='tab20',
        square=True,
        cbar_kws={'label': 'Class'},
        ax=axes[0]
    )
    axes[0].set_title('SOM Cell Labels', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('X')
    axes[0].set_ylabel('Y')
    
    # Reliability
    sns.heatmap(
        reliability,
        annot=True,
        fmt='.0f',
        cmap='YlOrRd',
        vmin=0,
        vmax=100,
        square=True,
        cbar_kws={'label': 'Reliability (%)'},
        ax=axes[1]
    )
    axes[1].set_title('SOM Cell Reliability', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('X')
    axes[1].set_ylabel('Y')
    
    plt.tight_layout()
    plt.show()


def plot_training_curves(history):
    """Plot training and validation curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train', marker='o')
    axes[0].plot(history['eval_loss'], label='Validation', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Loss Curves')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[1].plot(history['train_accuracy'], label='Train', marker='o')
    axes[1].plot(history['eval_accuracy'], label='Validation', marker='s')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Accuracy Curves')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_comparison(baseline_history, bpsom_history):
    """Plot comparison between baseline and BP-SOM."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Validation Loss
    axes[0].plot(baseline_history['eval_loss'], label='Baseline BERT', marker='o', linewidth=2)
    axes[0].plot(bpsom_history['eval_loss'], label='BP-SOM BERT', marker='s', linewidth=2)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Validation Loss', fontsize=12)
    axes[0].set_title('Comparison: Validation Loss', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)
    
    # Validation Accuracy
    axes[1].plot(baseline_history['eval_accuracy'], label='Baseline BERT', marker='o', linewidth=2)
    axes[1].plot(bpsom_history['eval_accuracy'], label='BP-SOM BERT', marker='s', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Validation Accuracy (%)', fontsize=12)
    axes[1].set_title('Comparison: Validation Accuracy', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("✓ Visualization functions defined")

## 6. Train Baseline BERT

First, train a baseline BERT model for comparison.

In [ ]:
# Configuration
num_epochs = 3
learning_rate = 2e-5
warmup_steps = 500

print("=" * 80)
print("BASELINE BERT EXPERIMENT")
print("=" * 80)

# Initialize baseline model
baseline_config = BertConfig.from_pretrained('bert-base-uncased')
baseline_config.num_labels = 2

baseline_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    config=baseline_config
).to(device)

# Add a dummy set_epoch method to the baseline model to satisfy the train_epoch function's expectation.
# The original BertForSequenceClassification does not have this method, but BPSOMBertForSequenceClassification does.
def dummy_set_epoch(self, epoch: int, max_epochs: int):
    pass
baseline_model.set_epoch = dummy_set_epoch.__get__(baseline_model, type(baseline_model))


# Optimizer and scheduler
baseline_optimizer = torch.optim.AdamW(baseline_model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
baseline_scheduler = get_linear_schedule_with_warmup(
    baseline_optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Training loop
baseline_history = {
    'train_loss': [],
    'train_accuracy': [],
    'eval_loss': [],
    'eval_accuracy': []
}

best_baseline_acc = 0.0

for epoch in range(num_epochs):
    # Train
    train_metrics = train_epoch(
        baseline_model,
        train_dataloader,
        baseline_optimizer,
        baseline_scheduler,
        device,
        epoch,
        num_epochs
    )

    # Evaluate
    eval_metrics = eval_epoch(baseline_model, eval_dataloader, device, "Validation")

    # Log
    print(f"\nEpoch {epoch+1}/{num_epochs}:")
    print(f"  Train - Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.2f}%")
    print(f"  Val   - Loss: {eval_metrics['loss']:.4f}, Acc: {eval_metrics['accuracy']:.2f}%")

    baseline_history['train_loss'].append(train_metrics['loss'])
    baseline_history['train_accuracy'].append(train_metrics['accuracy'])
    baseline_history['eval_loss'].append(eval_metrics['loss'])
    baseline_history['eval_accuracy'].append(eval_metrics['accuracy'])

    if eval_metrics['accuracy'] > best_baseline_acc:
        best_baseline_acc = eval_metrics['accuracy']
        print(f"  ✓ New best validation accuracy!")

    print("-" * 80)

print(f"\nBaseline Training Complete!")
print(f"Best Validation Accuracy: {best_baseline_acc:.2f}%")

In [ ]:
# Plot baseline results
plot_training_curves(baseline_history)

## 7. Train BP-SOM BERT

Now train the BP-SOM enhanced model.

In [ ]:
print("=" * 80)
print("BP-SOM BERT EXPERIMENT")
print("=" * 80)

# Initialize BP-SOM model
bpsom_config = BertConfig.from_pretrained('bert-base-uncased')
bpsom_config.num_labels = 2

bpsom_model_config = {
    'hidden_size': 128,
    'dropout': 0.1,
    'som_grid_size': 20,
    'som_error_weight': 0.25,
    'som_lr_max': 0.20,
    'som_lr_min': 0.05,
    'som_context_max': 2,
    'som_context_min': 0,
    'reliability_threshold': 0.95,
}

bpsom_model = BPSOMBertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    config=bpsom_config,
    bpsom_config=bpsom_model_config
).to(device)

# Optimizer and scheduler
bpsom_optimizer = torch.optim.AdamW(bpsom_model.parameters(), lr=learning_rate)
bpsom_scheduler = get_linear_schedule_with_warmup(
    bpsom_optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Training loop
bpsom_history = {
    'train_loss': [],
    'train_accuracy': [],
    'eval_loss': [],
    'eval_accuracy': [],
    'som_stats': []
}

best_bpsom_acc = 0.0

for epoch in range(num_epochs):
    # Reset SOM statistics
    bpsom_model.bpsom_hidden.som.reset_statistics()
    bpsom_model.bpsom_hidden.reset_activation_statistics()

    # Train
    train_metrics = train_epoch(
        bpsom_model,
        train_dataloader,
        bpsom_optimizer,
        bpsom_scheduler,
        device,
        epoch,
        num_epochs
    )

    # Update SOM cell labels
    # print("\nUpdating SOM cell labels...")
    bpsom_model.update_som_labels(train_dataloader, device)

    # Get SOM statistics
    som_stats = bpsom_model.bpsom_hidden.som.get_statistics()

    # Evaluate
    eval_metrics = eval_epoch(bpsom_model, eval_dataloader, device, "Validation")

    # Log
    print(f"\nEpoch {epoch+1}/{num_epochs}:")
    print(f"  Train - Loss: {train_metrics['loss']:.4f}, Acc: {train_metrics['accuracy']:.2f}%, "
          f"SOM Usage: {som_stats['som_usage_pct']:.1f}%")
    print(f"  Val   - Loss: {eval_metrics['loss']:.4f}, Acc: {eval_metrics['accuracy']:.2f}%")

    bpsom_history['train_loss'].append(train_metrics['loss'])
    bpsom_history['train_accuracy'].append(train_metrics['accuracy'])
    bpsom_history['eval_loss'].append(eval_metrics['loss'])
    bpsom_history['eval_accuracy'].append(eval_metrics['accuracy'])
    bpsom_history['som_stats'].append(som_stats)

    if eval_metrics['accuracy'] > best_bpsom_acc:
        best_bpsom_acc = eval_metrics['accuracy']
        print(f"  ✓ New best validation accuracy!")

    print("-" * 80)

print(f"\nBP-SOM Training Complete!")
print(f"Best Validation Accuracy: {best_bpsom_acc:.2f}%")

In [ ]:
# Plot BP-SOM results
plot_training_curves(bpsom_history)

## 8. Visualize SOM Organization

In [ ]:
# Visualize final SOM state
print("SOM Organization after Training:")
plot_som_visualization(bpsom_model, class_names=['negative', 'positive'])

## 9. Compare Results

In [ ]:
# Comparison plot
plot_comparison(baseline_history, bpsom_history)

# Summary statistics
print("\n" + "=" * 80)
print("FINAL COMPARISON SUMMARY")
print("=" * 80)
print(f"Baseline BERT:")
print(f"  Best Validation Accuracy: {max(baseline_history['eval_accuracy']):.2f}%")
print(f"  Final Validation Loss: {baseline_history['eval_loss'][-1]:.4f}")
print()
print(f"BP-SOM BERT:")
print(f"  Best Validation Accuracy: {max(bpsom_history['eval_accuracy']):.2f}%")
print(f"  Final Validation Loss: {bpsom_history['eval_loss'][-1]:.4f}")
print(f"  SOM Usage: {bpsom_history['som_stats'][-1]['som_usage_pct']:.1f}%")
print()

improvement = max(bpsom_history['eval_accuracy']) - max(baseline_history['eval_accuracy'])
print(f"Improvement: {improvement:+.2f}% accuracy")
print("=" * 80)

## 10. Analyze SOM Statistics

In [ ]:
# Plot SOM statistics over epochs
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

som_usage = [stat['som_usage_pct'] for stat in bpsom_history['som_stats']]
som_dist = [stat['avg_distance'] for stat in bpsom_history['som_stats']]

axes[0].plot(som_usage, marker='o', color='green', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('SOM Usage (%)', fontsize=12)
axes[0].set_title('SOM Error Usage Over Time', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

axes[1].plot(som_dist, marker='o', color='purple', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Average Distance', fontsize=12)
axes[1].set_title('SOM Average Distance Over Time', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSOM Statistics:")
print(f"  Final SOM Usage: {som_usage[-1]:.1f}%")
print(f"  Final Average Distance: {som_dist[-1]:.3f}")
print(f"  SOM Grid Size: {bpsom_model.bpsom_hidden.som.grid_size}x{bpsom_model.bpsom_hidden.som.grid_size}")

## 11. Save Models (Optional)

In [ ]:
# Uncomment to save models
# torch.save(baseline_model.state_dict(), 'baseline_bert_sst2.pt')
# torch.save(bpsom_model.state_dict(), 'bpsom_bert_sst2.pt')
# print("✓ Models saved")

## Conclusion

This notebook demonstrates BP-SOM applied to BERT fine-tuning on SST-2:

### Key Observations:

1. **SOM Organization**: The SOM develops class-specific clusters during training
2. **SOM Usage**: The percentage of examples where SOM error is used (reliability >= threshold)
3. **Performance**: Compare baseline vs BP-SOM accuracy

### Next Steps:

- Experiment with different SOM grid sizes
- Adjust SOM error weight (α)
- Try other GLUE tasks (MRPC, CoLA, etc.)
- Implement unit pruning
- Analyze learned representations

### References:

- Weijters, A. (1995). The BP-SOM architecture and learning algorithm. *Neural Processing Letters*, 2:6, pp. 13-16.
- Original C implementation: https://github.com/LanguageMachines/bp-som